In [66]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.losses import BinaryCrossentropy

In [67]:
G_train_df      = pd.read_csv ('data/preprocessed_data/G_train.csv')
T_train_df      = pd.read_csv ('data/preprocessed_data/T_train.csv')
G_T_train_df    = pd.read_csv ('data/preprocessed_data/balanced_G_T_train.csv')

G_cv_df         = pd.read_csv ('data/preprocessed_data/G_cv.csv')
T_cv_df         = pd.read_csv ('data/preprocessed_data/T_cv.csv')
G_T_cv_df        = pd.read_csv ('data/preprocessed_data/G_T_cv.csv')



In [68]:
random_state = 13
frac = 1

G_train_df      = G_train_df.sample(frac = frac, random_state=random_state)
T_train_df      = T_train_df.sample(frac = frac, random_state=random_state)
G_T_train_df    = G_T_train_df.sample(frac = frac, random_state=random_state)


In [69]:
G_T_train_df.shape

(128786, 3)

In [70]:
y_train = G_T_train_df.drop(columns = ['group ID','technique ID' ]).values
y_train.dtype

# G_train = sampled_G_train_df.drop(columns = ids)
G_train = G_train_df.drop(columns = 'group ID').values

# T_train = sampled_T_train_df.drop(columns = ids)
T_train = T_train_df.drop(columns = 'technique ID').values

```python
dataset = tf.data.Dataset.from_tensor_slices(({'input_Group': G_train_tf, 'input_Technique': T_train_tf}, y_train_tf))
```

1. `tf.data.Dataset`: This is a class in TensorFlow that represents a potentially large collection of elements. It's used to create pipelines for data processing and input to machine learning models.

2. `from_tensor_slices`: This is a method of the `tf.data.Dataset` class. It is used to create a dataset by slicing the given tensors along their first dimension (usually representing the number of samples). This is a convenient way to create a dataset where each element corresponds to a pair of inputs and their corresponding targets.

3. `({'input_Group': G_train_tf, 'input_Technique': T_train_tf}, y_train_tf)`: This is a tuple containing two elements. The first element is a dictionary that maps the input tensor names (`'input_Group'` and `'input_Technique'`) to their corresponding tensor data (`G_train_tf` and `T_train_tf`). The second element is the target tensor data `y_train_tf`. ❗The  names used in the from_tensor_slices dictionary should match the names of the input layers in your model. This ensures that the data from the dataset is correctly mapped to the corresponding input layers during training.


So, the overall process of this line of code is to create a dataset where each element is a pair of dictionaries (`{'input_Group': ..., 'input_Technique': ...}`) representing the input tensors and their corresponding target tensor.

When you iterate through this dataset, you'll get pairs like:
```
({'input_Group': G_train_sample, 'input_Technique': T_train_sample}, y_train_sample)
```

Here, `G_train_sample` and `T_train_sample` are individual samples from your `input_Group` and `input_Technique` tensors, and `y_train_sample` is the corresponding target value for that sample. This dataset structure is suitable for training machine learning models where you have multiple input features.

In [71]:
G_train_tf = tf.convert_to_tensor(G_train)
T_train_tf = tf.convert_to_tensor(T_train)
y_train_tf = tf.convert_to_tensor(y_train)
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_Group': G_train_tf, 'input_Technique': T_train_tf}, y_train_tf))

batch_size = 32


train_dataset = train_dataset.shuffle(buffer_size=len(G_train_tf))
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [72]:
y_cv = G_T_cv_df.drop(columns = ['group ID','technique ID' ]).values
y_cv.dtype

# G_cv = sampled_G_cv_df.drop(columns = ids)
G_cv = G_cv_df.drop(columns = 'group ID').values

# T_cv = sampled_T_cv_df.drop(columns = ids)
T_cv = T_cv_df.drop(columns = 'technique ID').values

In [73]:
G_cv_tf = tf.convert_to_tensor(G_cv)
T_cv_tf = tf.convert_to_tensor(T_cv)
y_cv_tf = tf.convert_to_tensor(y_cv)
cv_dataset = tf.data.Dataset.from_tensor_slices(({'input_Group': G_cv_tf, 'input_Technique': T_cv_tf}, y_cv_tf))
cv_dataset = cv_dataset.batch(batch_size)

---

In [74]:
# input shapes config
num_G_features = G_train.shape[1]  # remove Group ID during training
num_T_features = T_train.shape[1]   # remove Movie ID during training

# output
num_outputs = 32

tf.random.set_seed(random_state)

# Group NN
Group_NN = tf.keras.models.Sequential(
    layers=[
    tf.keras.layers.Dense (256, activation = 'relu'),
    tf.keras.layers.Dense (128, activation = 'relu'),
    tf.keras.layers.Dense (128, activation = 'relu'),
    tf.keras.layers.Dense (num_outputs, activation  = 'linear'),
    ], 
    name= "Group_NN")
# input vector for user_NN
input_Group = tf.keras.layers.Input(shape = (num_G_features), name = "input_Group")
vg = Group_NN(input_Group)
# vg = tf.linalg.l2_normalize(vg, axis=1)

# Technique NN
Technique_NN = tf.keras.models.Sequential(
    layers = [
    tf.keras.layers.Dense (256, activation = 'relu'),
    tf.keras.layers.Dense (128, activation = 'relu'),
    tf.keras.layers.Dense (128, activation = 'relu'),
    tf.keras.layers.Dense (num_outputs, activation  = 'linear'),  
    ],
    name = "Technique_NN")
# input vector for Technique_NN
input_Technique = tf.keras.layers.Input (shape= (num_T_features), name = "input_Technique")
vt = Technique_NN (input_Technique)
# vt = tf.linalg.l2_normalize (vt, axis = 1)

output = tf.keras.layers.Dot (axes=1)(inputs= [vg, vt])



In [75]:
tf.random.set_seed(random_state)
opt = keras.optimizers.Adam (learning_rate= 0.05)
model = tf.keras.Model (inputs = [input_Group, input_Technique],
                        outputs = output, name = 'recsysNN_model')
model.compile (optimizer = opt, loss = BinaryCrossentropy (from_logits= True))


## Early stopping

In [76]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss for early stopping
    patience=32,           # Number of epochs with no improvement before stopping
    restore_best_weights=True  # Restore model weights from the epoch with the best validation loss
)

In [77]:
epochs = 100
history = model.fit(train_dataset,
                    validation_data = cv_dataset,
                    epochs = epochs,
                    callbacks = [early_stopping])


Epoch 1/100
4025/4025 [==============================] - 20s 5ms/step - loss: 0.7559 - val_loss: 0.6931
Epoch 2/100
4025/4025 [==============================] - 20s 5ms/step - loss: 0.6942 - val_loss: 0.6434
Epoch 3/100
4025/4025 [==============================] - 20s 5ms/step - loss: 0.6949 - val_loss: 0.6944
Epoch 4/100
4025/4025 [==============================] - 21s 5ms/step - loss: 0.6951 - val_loss: 0.6963
Epoch 5/100
4025/4025 [==============================] - 20s 5ms/step - loss: 0.6951 - val_loss: 0.7370
Epoch 6/100
4025/4025 [==============================] - 20s 5ms/step - loss: 0.6951 - val_loss: 0.6717
Epoch 7/100
4025/4025 [==============================] - 20s 5ms/step - loss: 0.6948 - val_loss: 0.5160
Epoch 8/100
4025/4025 [==============================] - 21s 5ms/step - loss: 0.6952 - val_loss: 0.6977
Epoch 9/100
4025/4025 [==============================] - 20s 5ms/step - loss: 0.6948 - val_loss: 0.7600
Epoch 10/100
4025/4025 [==============================] - 20s 5m

In [78]:
import csv
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

training_loss_file = 'training_loss.csv'
validation_loss_file = 'validation_loss.csv'

with open(training_loss_file, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(training_loss)

with open(validation_loss_file, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(validation_loss)

In [79]:
print(len(training_loss))
print(len(validation_loss))

39
39


In [80]:
print (training_loss)
print (validation_loss)

[0.7559331059455872, 0.694195032119751, 0.6949310898780823, 0.6951173543930054, 0.695149838924408, 0.6950742602348328, 0.6947999000549316, 0.695155143737793, 0.6947545409202576, 0.6951875686645508, 0.694950520992279, 0.6951336860656738, 0.6952548027038574, 0.6947100758552551, 0.6951882243156433, 0.6951568722724915, 0.6948810815811157, 0.6949570178985596, 0.6954952478408813, 0.6951717138290405, 0.695451557636261, 0.6951914429664612, 0.6948603391647339, 0.695415198802948, 0.695213258266449, 0.6948020458221436, 0.694820761680603, 0.6949731111526489, 0.695512592792511, 0.6948270797729492, 0.6948840618133545, 0.6956643462181091, 0.694923460483551, 0.694814145565033, 0.695465087890625, 0.6949814558029175, 0.6948235630989075, 0.6950023770332336, 0.6950470805168152]
[0.6931461095809937, 0.6433700919151306, 0.6943687200546265, 0.6962643265724182, 0.7369785308837891, 0.6716542840003967, 0.5160380005836487, 0.6976915001869202, 0.7600456476211548, 0.6728299856185913, 0.6289705634117126, 0.71983599

In [81]:
# Plot the training and validation loss
# plt.plot(range(1, len(training_loss) + 1), training_loss, label='Training Loss')
# plt.plot(range(1, len(validation_loss) + 1), validation_loss, label='Validation Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

In [82]:
# plt.plot(range(1, len(training_loss) + 1), training_loss, label='Training Loss')